In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from matplotlib.widgets import Cursor
%matplotlib qt

In [2]:
data = pd.read_excel('15-Points.xlsx')

In [3]:
data

,x,y,City
0,5.500000e-08,9.860000e-09,1
1,-2.887330e+01,-7.980000e-08,2
2,-7.929160e+01,-2.140330e+01,3
3,-1.465770e+01,-4.338960e+01,4
4,-6.474730e+01,2.189820e+01,5
5,-2.905850e+01,-4.321670e+01,6
6,-7.207850e+01,1.815810e-01,7
7,-3.603660e+01,-2.161350e+01,8
8,-5.048080e+01,7.374470e+00,9
9,-5.058590e+01,-2.158820e+01,10


In [4]:
def load_data(data):
  data = [{'x' : x[0], 'y' : x[1], 'city' : x[2]} for x in data.values]
  return data

In [5]:
def Generate_distance(data):
  n = len(data)
  dist_mat = np.zeros((n, n))
  for i in range(n-1):
    x1 = data[i]['x']
    y1 = data[i]['y']
    dist_mat[i, i] = np.inf
    for j in range(i + 1, n):
      x2 = data[j]['x']
      y2 = data[j]['y']
      dist_mat[i, j] = sqrt((x1 - x2)**2 + (y1 - y2)**2)
      dist_mat[j, i] = dist_mat[i, j]
  dist_mat[i + 1, i + 1] = np.inf
  return dist_mat

In [6]:
def Initial_population(num_of_cities):
  population = [np.random.permutation(num_of_cities) for i in range(Population_size)]
  return np.array(population)

In [7]:
def Cost_Function(population, distances):
  J = np.zeros((len(population)))
  index = 0
  for Chromosome in population:
    dist = 0
    c0 = Chromosome[0]
    for i in range(len(Chromosome) - 1):
      c1 = Chromosome[i]
      c2 = Chromosome[i + 1]
      dist += distances[c1, c2]

    dist += distances[c0, c2]
    J[index] = dist
    index += 1
  return J

In [8]:
def Total_distance(Chromosome, distances):
  dist = 0
  c0 = Chromosome[0]
  for i in range(len(Chromosome) - 1):
      c1 = Chromosome[i]
      c2 = Chromosome[i + 1]
      dist += distances[c1, c2]

  dist += distances[c0, c2]
  return dist

In [9]:
def MultiCrossOver(Chromosome1, Chromosome2):

  Chromosome_size = (len(Chromosome1)) // 3
  
  Chromosome1_1 = Chromosome1[:Chromosome_size]
  Chromosome1_2 = Chromosome1[Chromosome_size : (Chromosome_size * 2)]
  Chromosome1_3 = Chromosome1[(Chromosome_size * 2):]

  Chromosome2_1 = Chromosome2[:Chromosome_size]
  Chromosome2_2 = Chromosome2[Chromosome_size:(Chromosome_size * 2)]
  Chromosome2_3 = Chromosome2[(Chromosome_size * 2):]

  next_gen1 = np.concatenate((Chromosome1_1, Chromosome2_2, Chromosome1_3))
  next_gen2 = np.concatenate((Chromosome2_1, Chromosome1_2, Chromosome2_3))
  return list(next_gen1), list(next_gen2)

In [10]:
def OnePointCrossOver(Chromosome1, Chromosome2):
  Chromosome_size = len(Chromosome1) - 1
  random_point = np.random.randint(0, Chromosome_size)
  
  Chromosome1_1 = Chromosome1[:random_point]
  Chromosome1_2 = Chromosome1[random_point:]

  Chromosome2_1 = Chromosome2[:random_point]
  Chromosome2_2 = Chromosome2[random_point:]

  next_gen1 = np.concatenate((Chromosome1_1, Chromosome2_2))
  next_gen2 = np.concatenate((Chromosome2_1, Chromosome1_2))
  return list(next_gen1), list(next_gen2)

In [11]:
def UniformCrossOver(Chromosome1, Chromosome2):
    new_gen1 = [None] * len(Chromosome1)
    new_gen2 = [None] * len(Chromosome2)

    for i in range(len(Chromosome1)):
        # flip a coin for each chromosome to decide weather or not it will be included in the off-spring
        if np.random.choice([True, False]):
            new_gen1[i] = Chromosome1[i]
            new_gen2[i] = Chromosome2[i]
        else:
            new_gen1[i] = Chromosome2[i]
            new_gen2[i] = Chromosome1[i]

    return new_gen1, new_gen2

In [12]:
def modified_uniform_crossover(Chromosome1, Chromosome2):    
    # create a mask to determine which cities to copy from Chromosome1
    mask = [np.random.choice([0, 1]) for _ in range(len(Chromosome1))]
    
    # create a set of cities already copied from Chromosome1
    copied_cities = [Chromosome1[i] for i in range(len(Chromosome1)) if mask[i] == 1]
    
    # create a list of remaining cities in the order of Chromosome2
    remaining_cities = [city for city in Chromosome2 if city not in copied_cities]
    
    # create the offspring by combining the copied cities and remaining cities
    offspring = [None] * len(Chromosome1)
    for i in range(len(Chromosome1)):
        if mask[i] == 1:
            offspring[i] = Chromosome1[i]
        else:
            offspring[i] = remaining_cities.pop(0)
    
    return offspring

In [13]:
def Swaping_Mutation(Chromosome):
  Chromosome_size = len(Chromosome)
  pos_to_swap = np.random.randint(0, Chromosome_size, 2)

  Chromosome[pos_to_swap[0]], Chromosome[pos_to_swap[1]] = Chromosome[pos_to_swap[1]], Chromosome[pos_to_swap[0]]
  return Chromosome

In [14]:
def ScrambleMutation(Chromosome):
    index1 = np.random.randint(len(Chromosome) - 1)
    index2 = np.random.randint(len(Chromosome) - 1)
    
    # Ensure index1 < index2
    if index1 > index2:
        index1, index2 = index2, index1
    
    # Scramble the values between index1 and index2
    scrambled_values = Chromosome[index1:index2+1]
    np.random.shuffle(scrambled_values)
    Chromosome[index1: index2 + 1] = scrambled_values
        
    return Chromosome

In [15]:
def InversionMutation(Chromosome):
    index1 = np.random.randint(len(Chromosome) - 1)
    index2 = np.random.randint(len(Chromosome) - 1)
    
    # Ensure index1 < index2
    if index1 > index2:
        index1, index2 = index2, index1
    
    # Select the subset between index1 and index2
    subset = Chromosome[index1: index2 + 1]
    inverted_values = np.flip(subset)
    Chromosome[index1: index2 + 1] = inverted_values
        
    return Chromosome

In [16]:
def ginput():
    points = []

    # display a plot and wait for the user to click on num_points points
    fig, ax = plt.subplots()
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.show()

    cursor = Cursor(ax, horizOn=True, vertOn=True, useblit=True, color='black', linewidth=1)

    # take input fro
    while True:
        # get the first tuple (first point)
        point = plt.ginput(1)
        if point:
            point=point[0]
        else:
            break
    #         print(point)
        points.append(point)
        plt.plot(point[0], point[1], 'bo')
        plt.draw()
    return points

In [17]:
def plot_tsp(data, Best_solution):
    x = [x['x'] for x in data]
    y = [x['y'] for x in data]
    plt.scatter(x, y, color = 'b')
    x_init, y_init = data[Best_solution[0]]['x'], data[Best_solution[0]]['y']
    plt.scatter(x_init, y_init, color = 'r')
    plt.text(x_init - 3, y_init - 4, "Start")

    x = []
    y = []
    x.append(x_init)
    y.append(y_init)
    TSP_plot = plt.plot(x, y, color = 'k')

    for i, city in enumerate(Best_solution[0:]):
        city_x, city_y = data[Best_solution[i]]['x'], data[Best_solution[i]]['y']
        x.append(city_x)
        y.append(city_y)  
        plt.setp(TSP_plot, xdata = x, ydata = y)
        plt.pause(0.3)

    x.append(x_init)
    y.append(y_init)
    plt.setp(TSP_plot, xdata = x, ydata = y)
    plt.pause(0.3)

In [18]:
Population_size = 50
Elitism_percentage = 0.05
CrossOver_Prop = 0.8
Mutation_Prop = 0.15
Generation_count = 150

In [19]:
def Genetic_Algorithm(data, Elitism_percentage, CrossOver_Prop, Mutation_Prop, Generation_count):
    
    # Load Data
    data = load_data(data)

    # Calculate the distances
    distances = Generate_distance(data)

    # Initialization: creating an initial population of potential solutions to the problem. Each solution is represented as a chromosome or a set of genes.
    population = Initial_population(len(data))

    # Evaluation: 
    J = Cost_Function(population, distances)

    # Selection:
    sorted_population = population[J.argsort()]

    # Calculate the number of elites, crossovers and mutations
    num_elites = round(Elitism_percentage * len(population))
    num_crossovers = round(CrossOver_Prop * (len(population) - num_elites))
    num_mutations = round(Mutation_Prop * (len(population) - num_elites))


    # Reproduction:
    for gen in range(Generation_count):
        next_generation = []
        
        # Choose the top solutions to the next generation via elitism
        for i in range(num_elites):
          next_generation.append(sorted_population[i])

        # Perform crossover on the remaining solutions

        for i in range(num_crossovers):
          inx1 = np.random.randint(len(sorted_population) - 1)
          inx2 = np.random.randint(len(sorted_population) - 1)
          parent1 = sorted_population[inx1]
          parent2 = sorted_population[inx2]

          child =  modified_uniform_crossover(parent1, parent2)
          next_generation.append(child)

        # Perform mutation on the next generation
        for i in range(num_mutations):
          inx = np.random.randint(len(sorted_population) - 1)
          parent = sorted_population[inx]

          Mutation_method = np.random.choice(['Swaping_Mutation', 'InversionMutation', 'ScrambleMutation'])
          if Mutation_method == 'Swaping_Mutation':
            child = Swaping_Mutation(parent)

          elif Mutation_method == 'InversionMutation':
            child = InversionMutation(parent)

          else:
            child = ScrambleMutation(parent)

          selection = [parent, child]
          next_generation.append(child)

        # Set the current population to the next generation
        next_generation = np.concatenate([next_generation, sorted_population]).reshape(-1, len(distances))

        # Calculate the cost function for each population
        J = Cost_Function(next_generation, distances)
        # Sort the population by J value
        sorted_population = next_generation[J.argsort()][:50]

    # Return the best solution found
    Best_solution = sorted_population[0]
    
    distance = Total_distance(Best_solution, distances)
    plot_tsp(data, Best_solution)
    
    return Best_solution, distance

In [20]:
Best_solution, distance = Genetic_Algorithm(data, Elitism_percentage, CrossOver_Prop, Mutation_Prop, Generation_count)

In [21]:
print(f'Best solution: {Best_solution} \nTotal distance: {distance}')

Best solution: [12  0 10  3  5  7  9 13 11  2  6  4  8 14  1] 
Total distance: 284.3810904080332


In [22]:
def Ginput_Genetic_Algorithm(Elitism_percentage, CrossOver_Prop, Mutation_Prop, Generation_count):
    
    points = ginput()
    data = pd.DataFrame(points, columns = ['x', 'y'])
    data['city'] = np.random.permutation(len(points))
    
    # Load Data
    data = load_data(data)

    # Calculate the distances
    distances = Generate_distance(data)

    # Initialization: creating an initial population of potential solutions to the problem. Each solution is represented as a chromosome or a set of genes.
    population = Initial_population(len(data))

    # Evaluation: 
    J = Cost_Function(population, distances)

    # Selection:
    sorted_population = population[J.argsort()]

    # Calculate the number of elites, crossovers and mutations
    num_elites = round(Elitism_percentage * len(population))
    num_crossovers = round(CrossOver_Prop * (len(population) - num_elites))
    num_mutations = round(Mutation_Prop * (len(population) - num_elites))


    # Reproduction:
    for gen in range(Generation_count):
        next_generation = []
        
        # Choose the top solutions to the next generation via elitism
        for i in range(num_elites):
          next_generation.append(sorted_population[i])

        # Perform crossover on the remaining solutions

        for i in range(num_crossovers):
          inx1 = np.random.randint(len(sorted_population) - 1)
          inx2 = np.random.randint(len(sorted_population) - 1)
          parent1 = sorted_population[inx1]
          parent2 = sorted_population[inx2]

          child =  modified_uniform_crossover(parent1, parent2)
          next_generation.append(child)

        # Perform mutation on the next generation
        for i in range(num_mutations):
          inx = np.random.randint(len(sorted_population) - 1)
          parent = sorted_population[inx]

          Mutation_method = np.random.choice(['Swaping_Mutation', 'InversionMutation', 'ScrambleMutation'])
          if Mutation_method == 'Swaping_Mutation':
            child = Swaping_Mutation(parent)

          elif Mutation_method == 'InversionMutation':
            child = InversionMutation(parent)

          else:
            child = ScrambleMutation(parent)

          selection = [parent, child]
          next_generation.append(child)

        # Set the current population to the next generation
        next_generation = np.concatenate([next_generation, sorted_population]).reshape(-1, len(distances))

        # Calculate the cost function for each population
        J = Cost_Function(next_generation, distances)
        # Sort the population by J value
        sorted_population = next_generation[J.argsort()][:50]

    # Return the best solution found
    Best_solution = sorted_population[0]
    
    distance = Total_distance(Best_solution, distances)
    plot_tsp(data, Best_solution)
    
    return Best_solution, distance

In [23]:
Best_solution, distance = Ginput_Genetic_Algorithm(Elitism_percentage, CrossOver_Prop, Mutation_Prop, Generation_count)
print(f'Best solution: {Best_solution} \nTotal distance: {distance}')

Best solution: [ 1 11 12  2  3  4  5  6  7  8  9 10 13  0] 
Total distance: 31.56527235295526
